In [1]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
tensorflow 2.6.2 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.2 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.
tensorflow 2.6.2 requires wrapt~=1.12.1, but you have wrapt 1.13.3 which is incompatible.
tensorflow-transform 1.5.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.
tensorflow-transform 1.5.0 requires numpy<1.20,>=1.16, but you have numpy 1.20.3 which is incompatible.
tensorflow-transform 1.5.0 requires pyarrow<6,>=1, but you have p

In [2]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

In [3]:
df = pd.read_csv('../input/news-summary/news_summary_more.csv')


In [4]:
df

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,"New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The match witnessed India getting all out for 92, their seventh lowest total in ODI cricket history."
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to â¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, customers have options to insure against Critical Illnesses, Disability and Accidental Death Benefit Rider with a life cover up to the age of 80 years."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""In the #MeToo movement, I always believe a woman. But in this case, we need to reserve our judgment,"" she added. Hirani has been accused by an assistant who worked in 'Sanju'."
...,...,...
98396,CRPF jawan axed to death by Maoists in Chhattisgarh,"A CRPF jawan was on Tuesday axed to death with sharp-edged weapons by Maoists at a local village fair in Chhattisgarh's insurgency-hit Bijapur district. As per preliminary information, Maoists attacked the jawan while he had gone to visit the fair along with his family, a police official said. A combing operation was launched to nab the assailants, he added."
98397,First song from Sonakshi Sinha's 'Noor' titled 'Uff Yeh' out,"'Uff Yeh', the first song from the Sonakshi Sinha starrer upcoming drama film 'Noor' has been released. The song has been composed by Amaal Mallik with vocals by Armaan Malik and lyrics by Manoj Muntashir. Sonakshi will be seen portraying a Pakistani journalist-writer in the film. Directed by Sunhil Sippy, the film is scheduled to release on April 21."
98398,'The Matrix' film to get a reboot: Reports,"According to reports, a new version of the 1999 science fiction film 'The Matrix' is in development. Michael B Jordan will reportedly play the lead role in the film. Screenwriter Zak Penn is in talks to write the script of the film, reports added. Actor Keanu Reeves starred in the original film, which was followed by two sequels."
98399,Snoop Dogg aims gun at clown dressed as Trump in new video,A new music video shows rapper Snoop Dogg aiming a toy gun at a clown character parodying US President Donald Trump. The video also shows a TV airing a news conference with the headline 'Ronald Klump wants to deport all doggs' airing live from 'The Clown House'. The video is for a remixed version of the song 'Lavender'.


In [5]:
df['text'] = df['text'].apply(lambda x: x.replace('/',''))
df['text'] = df['text'].apply(lambda x: x.replace('\xa0',''))
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,"New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The match witnessed India getting all out for 92, their seventh lowest total in ODI cricket history."
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to â¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, customers have options to insure against Critical Illnesses, Disability and Accidental Death Benefit Rider with a life cover up to the age of 80 years."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""In the #MeToo movement, I always believe a woman. But in this case, we need to reserve our judgment,"" she added. Hirani has been accused by an assistant who worked in 'Sanju'."


In [6]:
articles = df.head(17000)

In [7]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [8]:
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, task='summarization',
text_gen_kwargs={'max_length': 248,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False
                })

blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), get_y=ColReader('headlines'), splitter=RandomSplitter())

In [9]:
dls = dblock.dataloaders(articles, bs=2)

In [10]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [11]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... - done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=d351850d272ba7f5ecc933cbef7183e7146c4ea79d3929bded3fb7f342749f16
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
Initial GPU Usage
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
| ID | 

In [12]:
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.92k [00:00<?, ?B/s]

In [13]:
learn.fit_one_cycle(6, lr_max=3e-5, cbs=fit_cbs)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,1.160156,1.279690,0.333792,0.176708,0.282734,0.680722,0.798888,0.734496,47:47
1,1.006513,1.137135,0.415108,0.225713,0.360835,0.690787,0.809836,0.745109,44:11
2,0.705927,1.313555,0.382868,0.209821,0.332460,0.685425,0.808369,0.741144,48:27
3,0.449058,1.395029,0.381752,0.200704,0.329400,0.678466,0.805859,0.735866,49:36
4,0.303708,1.558812,0.387065,0.204282,0.332751,0.677806,0.804557,0.734964,49:59
5,0.234950,1.689660,0.384105,0.201231,0.329431,0.672386,0.803367,0.731055,53:43


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [14]:
df['text'][4]

'Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, "I\'ve known Hirani for many years...What if it\'s not true, the [#MeToo] movement will get derailed." "In the #MeToo movement, I always believe a woman. But in this case, we need to reserve our judgment," she added. Hirani has been accused by an assistant who worked in \'Sanju\'.'

In [15]:
outputs = learn.blurr_generate(df['text'][4], early_stopping=False, num_return_sequences=1)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
 If it's not true, #MeToo will get derailed: Sonam on Hirani's row  Â ÂI always believe a woman": Sonam KapoorÂ on charges against Hirani  â¹¹&#MeToo    'Sanju' maker accused of sexual harassment by assistant, Sonam defends Hirani



In [16]:
learn.metrics = None
learn.export(fname='best.pkl')

In [17]:
inf_learn = load_learner(fname='best.pkl')

In [18]:
test_article_1 = "Rezantsev was the commander of Russia's 49th combined army.A western official said he was the seventh general to die in Ukraine, and the second lieutenant general - the highest rank officer reportedly killed.It is thought that low morale among Russian troops has forced senior officers closer to the front line.In a conversation intercepted by the Ukrainian military, a Russian soldier complained that Rezantsev had claimed the war would be over within hours, just four days after it began.Ukrainian media reported on Friday that the general was killed at the Chornobaivka airbase near Kherson, which Russia is using as a command post and has been attacked by Ukraine's military several times.Another lieutenant general, Andrei Mordvichev, was reportedly killed by a Ukrainian strike on the same base.Kherson was the first Ukrainian city to be occupied by Russian forces, although there are reports that daily protests are held there against the Russian occupation.Although Russia has confirmed the death of only one general, Kyiv and western officials believe up to seven have been killed in fighting since the war began.However the death of Maj Gen Magomed Tushayev of the Chechen national guard has been disputed.It is unusual for such senior Russian officers to be so close to the battlefield, and western officials believe that they have been forced to move towards the front lines to deal with low morale among Russian troops.The unexpectedly strong Ukrainian resistance, poor Russian equipment and a high death toll amongst Russian troops are all thought to be contributing to the low morale.Russian forces are believed to be relying in part on open communication systems, for example mobile phones and analogue radios, which are easy to intercept and could give away the locations of high-ranking officers.A person inside Ukrainian President Volodymyr Zelensky's inner circle told the Wall Street Journal that Ukraine had a military intelligence team dedicated to targeting Russia's officer class.So far, Vladimir Putin has only referred to the death of one general, thought to be Maj Gen Andrey Sukhovetsky, in a speech soon after the start of the war.Russia says 1,351 soldiers have died since the war began in Ukraine, although Kyiv and western officials say the number is much higher."

In [19]:
test_article_2 = "A team of officers of the Central Bureau of Investigation (CBI) on Saturday visited Bogtui village in West Bengal ‘s Birbhum district where eight persons were burnt to death on March 21 following the murder of a local Trinamool Congress leader. The Calcutta High Court on Friday directed the CBI to probe the violence in the “interest of justice”, “fair investigation” and to “instill confidence in society”.During the day, CBI officers led by DIG Akhilesh Singh visited the village and inspected the houses that were set on fire. The CBI team comprising over a dozen personnel went to the house of Sona Sheikh where seven persons were charred to death.The investigating officers went to the room from where the bodies were recovered and also to the terrace of the house. They also visited the other houses that were set on fire and used a digital three– dimensional scanning device to collect evidence.Forensic experts collected samples from the site for the second consecutive day. While the investigators were collecting evidence, TMC Rampurhat Block–I president Anaraul Hossain, who was arrested on the instructions of Chief Minister Mamata Banerjee, said “there has been a conspiracy” against him. The TMC leader, who was being taken to hospital by the police, told the media that he was at his house on the night of March 21 when the violence occurred and the police did not call him. The Chief Minister had blamed the local leader along with the Rampurhat Sub-Divisional Police Office for the violence. The SDPO has been removed and sent on compulsory waiting."

In [20]:
test_article_3 = """The defence ministry said that the initial aims of the war were complete, and that Russia had reduced the combat capacity of Ukraine.Russia's invasion appeared aimed at swiftly capturing major cities and toppling the government.But it has stalled in the face of fierce Ukrainian resistance."The main tasks of the first stage of the operation have been carried out," said Sergei Rudskoy, head of the General Staff's main operations administration."The combat capabilities of the Ukrainian armed forces have been substantially reduced, which allows us to concentrate our main efforts on achieving the main goal: the liberation of Donbas," he added, referring to an area in eastern Ukraine largely in the hands of Russian-backed separatists.Russia's military has been bombarding and trying to encircle key Ukrainian cities such as the capital Kyiv, which Gen Rudskoy characterised as an attempt to tie down Ukraine's forces elsewhere in the country while Russia focuses on the east.Ukraine's President, Volodymyr Zelensky, said his troops had landed "powerful blows" on Russia and called on Moscow to recognise the need for serious peace talks."By restraining Russia's actions, our defenders are leading the Russian leadership to a simple and logical idea: talk is necessary. Meaningful. Urgent. Fair. For the sake of the result, not for the sake of the delay," he said."""

In [21]:
test_article_4 = """ Leaked papers indicate a Chinese military base could potentially be set up on the island to Australia's north.That's sparked concern from Australia, long the chief defence partner and biggest aid donor to the tiny island.Both Australia and New Zealand said it had raised concerns with Honiara.Australia's Foreign Minister Marise Payne said she respected the Pacific island's right to make sovereign decisions but:"We would be particularly concerned by any actions that undermine the stability and security of our region, including the establishment of a permanent presence such as a military base."New Zealand said it was also concerned as the plan threatened to "destabilise the current institutions and arrangements that have long underpinned the Pacific region's security." """

In [22]:
inf_learn.blurr_generate(test_article_1)

[" 7th Russian general killed in Ukraine war, 2nd top-ranking officer killed: Report    Russia says 1,351 soldiers killed in total but rebels say more than 1,000Â\xa0(Ukrainian claims disputed) Russian troops' low morale blamed for high death toll"]

In [23]:
inf_learn.blurr_generate(test_article_2)

[' CBI officers visit Bogtui village where 8 died in TMC violence in WB in March 21Â\xa0& 2nd timeÂ\xa0in a dayÂ\xa0on same dayÂÂ\x80\x99s still 8 bodies foundÂ\xa0unharmedÂ\xa0']

In [24]:
inf_learn.blurr_generate(test_article_3)

[" Initial aims of war complete, says Russia as Ukraine holds firm: Russia Defence Ministry Â\xa0Russia invaded Ukraine to topple govt, stalledÂ\xa0in face of resistance Â\x80\x98Putin says 'talk' not 'waiting' for 'result'"]

In [25]:
inf_learn.blurr_generate(test_article_4)

[" Leaked Chinese military plan to set up base near Australia concerns us: NZ & Australia  Â\x80\x98Concerned' Â\xa0for Pacific island Âº&NEW  Tip of the hat to Australia, its biggest aid donor irc among many nations"]